# Member 1 Implementation Verification

## Core RL & Environment

This notebook verifies that **Member 1**'s implementation is correct and complete. Member 1 is responsible for:

1. **CustomHopper** (`env/custom_hopper.py`): Environment extension with ADR support
2. **ADRCallback** (`callbacks/adr_callback.py`): Callback for adaptive difficulty control

Each section of this notebook tests a specific component with clear success or failure messages.

---

## 1. Setup and Imports

First, let's verify that all dependencies are installed and that the project modules can be imported correctly.

In [ ]:
import sys
import os

# Add project root to path for imports
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Project root: {project_root}")

In [ ]:
# Test import of main dependencies
import numpy as np
import gymnasium as gym

print("[OK] numpy imported successfully")
print("[OK] gymnasium imported successfully")

---

## 2. CustomHopper Verification

### 2.1 Module Import

The first test verifies that the `custom_hopper.py` file can be imported without syntax errors.

In [ ]:
try:
    from env.custom_hopper import CustomHopper
    print("[OK] CustomHopper imported successfully")
except ImportError as e:
    print(f"[ERROR] Cannot import CustomHopper: {e}")
except Exception as e:
    print(f"[ERROR] Error during import: {e}")

### 2.2 Environment Registration

Let's verify that the custom environments are correctly registered in Gymnasium.

In [ ]:
# List of environments that should be registered
expected_envs = [
    'CustomHopper-v0',
    'CustomHopper-source-v0',
    'CustomHopper-target-v0'
]

all_registered = True
for env_id in expected_envs:
    if env_id in gym.envs.registry:
        print(f"[OK] {env_id} is registered")
    else:
        print(f"[ERRORE] {env_id} NON is registered")
        all_registered = False

if all_registered:
    print("\n[SUCCESS] All environments are correctly registered!")

### 2.3 Environment Creation

Let's test creating an instance of the CustomHopper environment.

In [ ]:
try:
    env = gym.make('CustomHopper-source-v0', udr=False)
    print("[OK] Environment created successfully")
    print(f"    Observation space: {env.observation_space}")
    print(f"    Action space: {env.action_space}")
except Exception as e:
    print(f"[ERROR] Cannot create environment: {e}")
    env = None

### 2.4 ADR Attributes Verification

Member 1 should have added the necessary ADR attributes in the class `__init__`.

In [ ]:
if env is not None:
    env_unwrapped = env.unwrapped
    
    # List of required ADR attributes
    required_attrs = [
        ('original_masses', 'Original masses saved'),
        ('original_damping', 'Original damping saved'),
        ('original_friction', 'Original friction saved'),
        ('adr_state', 'ADR state (dictionary)'),
        ('adr_steps_size', 'Step size for ADR'),
        ('min_friction_floor', 'Minimum floor for friction')
    ]
    
    all_present = True
    print("ADR attributes verification:\n")
    
    for attr, description in required_attrs:
        if hasattr(env_unwrapped, attr):
            value = getattr(env_unwrapped, attr)
            if isinstance(value, np.ndarray):
                print(f"[OK] {attr}: array shape {value.shape}")
            elif isinstance(value, dict):
                print(f"[OK] {attr}: {value}")
            else:
                print(f"[OK] {attr}: {value}")
        else:
            print(f"[ERRORE] {attr} MISSING - {description}")
            all_present = False
    
    if all_present:
        print("\n[SUCCESS] All ADR attributes are present!")
else:
    print("[SKIP] Environment not available")

### 2.5 adr_state Verification

The `adr_state` dictionary should have the correct keys and initial values at zero.

In [ ]:
if env is not None and hasattr(env.unwrapped, 'adr_state'):
    adr_state = env.unwrapped.adr_state
    
    required_keys = ['mass_range', 'damping_range', 'friction_range']
    
    print("adr_state structure verification:\n")
    
    all_correct = True
    for key in required_keys:
        if key in adr_state:
            value = adr_state[key]
            if value == 0.0:
                print(f"[OK] '{key}': {value} (correct, starts at 0)")
            else:
                print(f"[WARNING] '{key}': {value} (should be 0.0 initially)")
        else:
            print(f"[ERRORE] Key '{key}' MISSING in adr_state")
            all_correct = False
    
    if all_correct:
        print("\n[SUCCESS] adr_state is correctly structured!")
else:
    print("[SKIP] adr_state not available")

### 2.6 ADR Methods Verification

Member 1 should have implemented the `update_adr()` and `get_adr_info()` methods.

In [ ]:
if env is not None:
    env_unwrapped = env.unwrapped
    
    required_methods = [
        ('update_adr', 'Updates the ADR state based on reward'),
        ('get_adr_info', 'Returns the current ADR state'),
        ('sample_parameters', 'Samples physical parameters'),
        ('set_parameters', 'Applies physical parameters')
    ]
    
    print("ADR methods verification:\n")
    
    all_present = True
    for method, description in required_methods:
        if hasattr(env_unwrapped, method) and callable(getattr(env_unwrapped, method)):
            print(f"[OK] {method}() present")
        else:
            print(f"[ERRORE] {method}() MISSING - {description}")
            all_present = False
    
    if all_present:
        print("\n[SUCCESS] All ADR methods are present!")
else:
    print("[SKIP] Environment not available")

### 2.7 Functional Test: update_adr()

Let's test that `update_adr()` works correctly by simulating the three scenarios: expansion, contraction, and stability.

In [ ]:
if env is not None and hasattr(env.unwrapped, 'update_adr'):
    env_unwrapped = env.unwrapped
    
    # Reset adr_state for the test
    env_unwrapped.adr_state = {
        "mass_range": 0.1,
        "damping_range": 0.1,
        "friction_range": 0.1
    }
    
    print("Testing update_adr():\n")
    
    # Test 1: Expansion (reward alto)
    try:
        result = env_unwrapped.update_adr(mean_reward=2500, low_th=1000, high_th=2000)
        if result is not None:
            status, state = result
            if status == "expanded":
                print(f"[OK] Expansion: reward=2500 -> status='{status}'")
                print(f"    Nuovo mass_range: {state.get('mass_range', 'N/A')}")
            else:
                print(f"[WARNING] Expected 'expanded', got '{status}'")
        else:
            print("[ERRORE] update_adr() returned None - method not implemented")
    except Exception as e:
        print(f"[ERRORE] Exception during expansion test: {e}")
    
    # Test 2: Contraction (reward basso)
    try:
        result = env_unwrapped.update_adr(mean_reward=500, low_th=1000, high_th=2000)
        if result is not None:
            status, state = result
            if status == "contracted":
                print(f"[OK] Contraction: reward=500 -> status='{status}'")
                print(f"    Nuovo mass_range: {state.get('mass_range', 'N/A')}")
            else:
                print(f"[WARNING] Expected 'contracted', got '{status}'")
        else:
            print("[ERRORE] update_adr() returned None - method not implemented")
    except Exception as e:
        print(f"[ERRORE] Exception during contraction test: {e}")
    
    # Test 3: Stability (reward medio)
    try:
        result = env_unwrapped.update_adr(mean_reward=1500, low_th=1000, high_th=2000)
        if result is not None:
            status, state = result
            if status == "stable":
                print(f"[OK] Stability: reward=1500 -> status='{status}'")
            else:
                print(f"[WARNING] Expected 'stable', got '{status}'")
        else:
            print("[ERRORE] update_adr() returned None - method not implemented")
    except Exception as e:
        print(f"[ERRORE] Exception during stability test: {e}")
else:
    print("[SKIP] update_adr not available")

### 2.8 Functional Test: sample_parameters()

Let's verify that `sample_parameters()` returns a dictionary with the correct keys (ADR version) or a list (legacy UDR version).

In [ ]:
if env is not None and hasattr(env.unwrapped, 'sample_parameters'):
    env_unwrapped = env.unwrapped
    
    print("Testing sample_parameters():\n")
    
    try:
        params = env_unwrapped.sample_parameters()
        
        if isinstance(params, dict):
            print("[OK] sample_parameters() returns a dictionary (ADR version)")
            
            required_keys = ['masses', 'damping', 'friction']
            for key in required_keys:
                if key in params:
                    print(f"    [OK] '{key}' present, shape: {np.array(params[key]).shape}")
                else:
                    print(f"    [ERRORE] '{key}' MISSING nel dizionario")
        elif isinstance(params, list):
            print("[WARNING] sample_parameters() returns a list (legacy UDR version)")
            print(f"    Number of masses: {len(params)}")
            print("    -> Update to ADR version that returns a dict")
        else:
            print(f"[WARNING] Unexpected type: {type(params)}")
            
    except Exception as e:
        print(f"[ERRORE] Exception during sample_parameters(): {e}")
else:
    print("[SKIP] sample_parameters not available")

### 2.9 Functional Test: Environment Step

Let's verify that the environment works correctly after a reset and some steps.

In [ ]:
if env is not None:
    print("Environment functional test:\n")
    
    try:
        obs, info = env.reset()
        print(f"[OK] Reset completed, obs shape: {obs.shape}")
        
        total_reward = 0
        for i in range(10):
            action = env.action_space.sample()
            obs, reward, terminated, truncated, info = env.steps(action)
            total_reward += reward
            
            if terminated:
                print(f"[INFO] Episode terminated after {i+1} steps")
                break
        
        print(f"[OK] {i+1} steps completati, reward totale: {total_reward:.2f}")
        print("\n[SUCCESSO] L'ambiente funziona correttamente!")
        
    except Exception as e:
        print(f"[ERRORE] Exception during test: {e}")
else:
    print("[SKIP] Environment not available")

---

## 3. ADRCallback Verification

### 3.1 Module Import

In [ ]:
try:
    from callbacks.adr_callback import ADRCallback
    print("[OK] ADRCallback imported successfully")
except ImportError as e:
    print(f"[ERROR] Cannot import ADRCallback: {e}")
except Exception as e:
    print(f"[ERROR] Error during import: {e}")

### 3.2 Class Structure Verification

Let's verify that ADRCallback has the required methods and attributes.

In [ ]:
try:
    from callbacks.adr_callback import ADRCallback
    from stable_baselines3.common.callbacks import BaseCallback
    
    print("ADRCallback structure verification:\n")
    
    # Verify inheritance
    if issubclass(ADRCallback, BaseCallback):
        print("[OK] ADRCallback inherits from BaseCallback")
    else:
        print("[ERROR] ADRCallback does not inherit from BaseCallback")
    
    # Create instance for testing
    callback = ADRCallback(check_freq=2048)
    
    # Verify attributes
    required_attrs = ['check_freq', 'threshold_high', 'threshold_low']
    for attr in required_attrs:
        if hasattr(callback, attr):
            print(f"[OK] Attribute '{attr}': {getattr(callback, attr)}")
        else:
            print(f"[ERRORE] Attribute '{attr}' MISSING")
    
    # Verify _on_step methods
    if hasattr(callback, '_on_steps') and callable(callback._on_steps):
        print("[OK] Metodo _on_steps() present")
    else:
        print("[ERRORE] Metodo _on_steps() MISSING")
        
except Exception as e:
    print(f"[ERRORE] Exception during verification: {e}")

### 3.3 Performance Thresholds Verification

The thresholds should be reasonable for Hopper (solved ~3000).

In [ ]:
try:
    from callbacks.adr_callback import ADRCallback
    
    callback = ADRCallback(check_freq=2048)
    
    print("Performance thresholds verification:\n")
    
    high = callback.threshold_high
    low = callback.threshold_low
    
    print(f"Threshold High: {high}")
    print(f"Threshold Low: {low}")
    
    if low < high:
        print("[OK] low < high (correct)")
    else:
        print("[ERRORE] low should be < high")
    
    if 500 <= low <= 1500:
        print("[OK] low is in a reasonable range (500-1500)")
    else:
        print(f"[WARNING] low={low} might not be optimal")
    
    if 1500 <= high <= 2500:
        print("[OK] high is in a reasonable range (1500-2500)")
    else:
        print(f"[WARNING] high={high} might not be optimal")
        
except Exception as e:
    print(f"[ERRORE] Exception during verification: {e}")

---

## 4. Final Summary

Let's run a summary of all tests for an overview.

In [ ]:
print("="*60)
print("MEMBER 1 VERIFICATION SUMMARY")
print("="*60)

checks = []

# Check CustomHopper
try:
    from env.custom_hopper import CustomHopper
    checks.append(("Import CustomHopper", True))
except:
    checks.append(("Import CustomHopper", False))

# Check registrazione ambienti
import gymnasium as gym
env_registered = 'CustomHopper-source-v0' in gym.envs.registry
checks.append(("Environment registration", env_registered))

# Check attributi ADR
try:
    env = gym.make('CustomHopper-source-v0', udr=False)
    has_adr = hasattr(env.unwrapped, 'adr_state')
    checks.append(("ADR attributes", has_adr))
    env.close()
except:
    checks.append(("ADR attributes", False))

# Check metodi ADR
try:
    env = gym.make('CustomHopper-source-v0', udr=False)
    has_update = hasattr(env.unwrapped, 'update_adr')
    checks.append(("update_adr method", has_update))
    env.close()
except:
    checks.append(("update_adr method", False))

# Check ADRCallback
try:
    from callbacks.adr_callback import ADRCallback
    checks.append(("Import ADRCallback", True))
except:
    checks.append(("Import ADRCallback", False))

# Print results
passed = 0
for name, result in checks:
    status = "PASS" if result else "FAIL"
    symbol = "✓" if result else "✗"
    print(f"  [{symbol}] {name}: {status}")
    if result:
        passed += 1

print("\n" + "="*60)
print(f"TOTAL: {passed}/{len(checks)} tests passed")
print("="*60)

if passed == len(checks):
    print("\n[SUCCESS] Member 1 implementation COMPLETE!")
else:
    print(f"\n[WARNING] {len(checks) - passed} tests failed. Review the implementation.")

---

## 5. Cleanup

In [ ]:
# Close any open environments
try:
    env.close()
    print("Environment closed successfully.")
except:
    pass